In [12]:
import pandas as pd
from bs4 import BeautifulSoup
import re
from selenium import webdriver
import chromedriver_binary
import string
import numpy as np

pd.options.display.float_format = '{:.0f}'.format

In [13]:
ticker = input("Input the ticker of the company you'd like to see the financials of: ")

is_link = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
bs_link = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
cf_link = f'https://finance.yahoo.com/quote/{ticker}/cash-flow?p={ticker}'

Input the ticker of the company you'd like to see the financials of: TSLA


In [23]:
def yahoo_financial_statements():
    
    ticker = input("Input the ticker of the company you'd like to see the financials of: ")

    is_link = f'https://finance.yahoo.com/quote/{ticker}/financials?p={ticker}'
    bs_link = f'https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}'
    cf_link = f'https://finance.yahoo.com/quote/{ticker}/cash-flow?p={ticker}'
    
    statements_list = [is_link,bs_link,cf_link]
    
    headers = []
    temp_list = []
    label_list = []
    final = []
    index = 0
    
    df_lists = list()
    # comment 3 lines below if you're not using Opera
    options = webdriver.ChromeOptions()
    options.add_argument('allow-elevated-browser')
    options.add_experimental_option('w3c', True)
    #driver = webdriver.Chrome() # uncomment if you're not using Opera
    driver = webdriver.Opera(executable_path="C:\\Users\\chris\\Downloads\\operadriver_win64\\operadriver.exe", options=options) # comment if you're not using Opera
    
    for link in statements_list:
        
        
        driver.get(link)
        html = driver.execute_script('return document.body.innerHTML;')
        soup = BeautifulSoup(html,'lxml')
        
        features = soup.find_all('div', class_='D(tbr)')
        
        #create headers
        for item in features[0].find_all('div', class_='D(ib)'):
            headers.append(item.text)
            
        #statement contents
        while index <= len(features)-1:
            #filter for each line of the statement
            temp = features[index].find_all('div', class_='D(tbc)')
            for line in temp:
                #each item added to a temp list
                temp_list.append(line.text)
            #temp_list added to final list
            final.append(temp_list)
            #clear temp_list
            temp_list = []
            index+=1
        
        #print("Final: ", final)
        #print("Headers: ", headers)
        
        df = pd.DataFrame(final[1:])
        df.columns = headers
        df.index = np.transpose(final[1:]).tolist()[0]
        
        #function to make all values numerical
        def convert_to_numeric(column):

            first_col = [i.replace(',','') for i in column]
            second_col = [i.replace('-','') for i in first_col]
            final_col = pd.to_numeric(second_col)

            return final_col
        
        for column in headers[1:]:
            df[column] = convert_to_numeric(df[column])
        
        final_df = df.fillna('-')
        df_lists.append(final_df)
        
        #reset all lists
        headers = []
        temp_list = []
        label_list = []
        final = []
        index = 0

    return df_lists
        

In [24]:
financials = yahoo_financial_statements()

Input the ticker of the company you'd like to see the financials of: AAPL


In [25]:
income_statement = financials[0]
balance_sheet = financials[1]
cash_flow_statement = financials[2]

In [26]:
income_statement.index = income_statement[['Breakdown']]

In [27]:
income_statement

,Breakdown,ttm,9/30/2022,9/30/2021,9/30/2020,9/30/2019
"(Total Revenue,)",Total Revenue,387537000,394328000,365817000,274515000,260174000
"(Cost of Revenue,)",Cost of Revenue,220666000,223546000,212981000,169559000,161782000
"(Gross Profit,)",Gross Profit,166871000,170782000,152836000,104956000,98392000
"(Operating Expense,)",Operating Expense,52906000,51345000,43887000,38668000,34462000
"(Operating Income,)",Operating Income,113965000,119437000,108949000,66288000,63930000
"(Net Non Operating Interest Income Expense,)",Net Non Operating Interest Income Expense,197000,106000,198000,890000,1385000
"(Other Income Expense,)",Other Income Expense,283000,228000,60000,87000,422000
"(Pretax Income,)",Pretax Income,113485000,119103000,109207000,67091000,65737000
"(Tax Provision,)",Tax Provision,18314000,19300000,14527000,9680000,10481000
"(Net Income Common Stockholders,)",Net Income Common Stockholders,95171000,99803000,94680000,57411000,55256000


In [28]:
income_statement.iloc[0,1]

387537000.0

In [29]:
income_statement.iloc[0][2]

394328000.0

In [30]:
income_statement[['Breakdown']].iloc[0]

Breakdown    Total Revenue
Name: (Total Revenue,), dtype: object